In [ ]:
MATCH (a:Author)-[:AUTHORED]->(:Quanta)-[m:METRICS_IN]->(:Year {year: 2019})
WITH m
ORDER BY m.timeScaledPageRank DESC
with collect(m) as scores
with scores, size(scores) as n
match (a2:Author)-[:AUTHORED]->(:Quanta)-[m2:METRICS_IN]->(:Year {year: 2019})
where a2.cleanName in
	['piyushgupta', 'philstephens', 'thomasliem',
	'stephenpnaber', 'philipghaydon', 'barrymberger',
	'bhishamchera', 'gaoravpgupta']
with 
	a2.cleanName as name,
	m2.timeScaledPageRank as tspr, 
	size([x in scores where m2.timeScaledPageRank < x.timeScaledPageRank | x])/size(scores) as rank
return 
	name, avg(rank) as avgRank, max(rank) as maxRank, percentileCont(rank, 0.5) as medianRank